In [1]:
import yfinance as yf
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
from datetime import datetime, timedelta

In [3]:
stocks = [
            {'label': 'Apple (AAPL)', 'value': 'AAPL'},
            {'label': 'Tesla (TSLA)', 'value': 'TSLA'},
            {'label': 'Amazon (AMZN)', 'value': 'AMZN'},
        ]

def fetch_stocks():
    data = {}

    for i in stocks:
        data[i['value']] = yf.download(i['value'], start='2023-10-12', end='2024-10-12')
    return data

In [5]:
app = dash.Dash(__name__)

data = fetch_stocks()
app.layout = html.Div([
    html.H1("Stocks on Candles"),
    
    # Stock selection dropdown
    dcc.Dropdown(
        id='stock-selector',
        options=stocks,
        value='AAPL',
        style={'width': '200px', 'margin-bottom': '20px'}
    ),
    
    # Adding Candlestick chart to dash
    dcc.Graph(id='candlestick-chart'),
    
    # Date range slider
    dcc.RangeSlider(
        id='date-range-slider',
        min=0,
        max=100,
        step=1,
        value=[0, 100],
        marks={},
        allowCross=False
    )
])

@app.callback(
    [Output('candlestick-chart', 'figure'),
     Output('date-range-slider', 'max'),
     Output('date-range-slider', 'marks'),
     Output('date-range-slider', 'value')],
    [Input('stock-selector', 'value'),
     Input('date-range-slider', 'value')]
)
def update_chart(selected_stock, selected_range):
    global data
    df = data[selected_stock]

    slider_max = len(df) - 1
    slider_marks = slider_marks = {i: df.index[i].strftime('%Y-%m-%d') \
                                   for i in range(0, len(df), max(1, len(df) // 10))}
    
    start_idx, end_idx = selected_range
    df_filtered = df.iloc[start_idx:end_idx+1]
    
    candlestick = go.Candlestick(
        x=df_filtered.index,
        open=df_filtered['Open'],
        high=df_filtered['High'],
        low=df_filtered['Low'],
        close=df_filtered['Close']
    )
    
    y_min = df_filtered['Low'].min()
    y_max = df_filtered['High'].max()
    y_range = [y_min - (y_max - y_min) * 0.1, y_max + (y_max - y_min) * 0.1]
    
    fig = go.Figure(data=[candlestick])
    fig.update_layout(
        title=f'{selected_stock} Stock Price',
        height= 600,
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        yaxis_range=y_range
    )
    slider_value = [start_idx, max(end_idx, slider_max)]
    
    return fig, slider_max, slider_marks, slider_value

if __name__ == '__main__':
    app.run_server(port=3000)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
